In [ ]:
import time
import os
import json

from pydantic import BaseModel, Field
from google import genai
from google.genai import types

INPUT_FILE = "math_dataset_with_answers.jsonl"
OUTPUT_FILE = "incorrect_analysis.json"
MODEL_NAME = "gemini-2.5-pro" 
BATCH_SIZE = 100  # 每批处理的数量

#aistudio.google.com
API_KEY="你自己的KEY"

# ================= 数据结构定义 =================
class IncorrectItem(BaseModel):
    id: int = Field(description="题目对应的 ID 编号")
    reason: str = Field(description="简短解释为什么这道题是错的（例如：'计算错误，3+5应为8' 或 '逻辑推导有误'）")

class ReviewResult(BaseModel):
    incorrect_items: list[IncorrectItem]

# ================= 核心功能函数 =================

def process_batch_with_retry(client, batch_data, batch_index, max_retries=3):
    """
    处理单个批次，包含重试逻辑
    """
    # 1. 构造当前批次的 Prompt
    batch_content_str = ""
    for item in batch_data:
        record_str = (
            f"ID: {item['original_id']}\n"
            f"Question: {item['data'].get('instruction')}\n"
            f"Thinking: {item['data'].get('thinking')}\n"
            f"Answer: {item['data'].get('output')}\n"
            f"-----------------------------\n"
        )
        batch_content_str += record_str

    prompt = (
        "你是一位资深数学老师。请仔细检查以下数学题的【推理过程】和【最终答案】。\n"
        "任务要求：\n"
        "1. 如果题目完全正确，忽略它。\n"
        "2. 如果有错，请记录其 ID，并用简短的中文说明错误原因。\n"
        "3. 严格按照定义的 JSON 格式返回结果。\n"
        "\n"
        "以下是本批次待检查的题目：\n"
        f"{batch_content_str}"
    )

    # 2. 重试循环
    wait_time = 5 # 初始等待 5 秒
    
    for attempt in range(max_retries):
        try:
            response = client.models.generate_content(
                model=MODEL_NAME,
                contents=prompt,
                config=types.GenerateContentConfig(
                    response_mime_type="application/json",
                    response_schema=ReviewResult,
                    temperature=0.1,
                ),
            )
            
            if response.parsed:
                return response.parsed.incorrect_items
            else:
                print(f"   ⚠️ 第 {batch_index} 批返回无法解析，尝试重试...")
                
        except Exception as e:
            # 捕获错误（包括 503 Overloaded）
            print(f"   ⚠️ 第 {batch_index} 批发生错误 (尝试 {attempt+1}/{max_retries}): {e}")
            if "503" in str(e) or "429" in str(e):
                print(f"      服务器繁忙，休息 {wait_time} 秒...")
                time.sleep(wait_time)
                wait_time *= 2 # 指数退避：5s -> 10s -> 20s
            else:
                # 如果是其他严重错误，可能不需要重试，视情况而定
                time.sleep(2)

    print(f"   ❌ 第 {batch_index} 批彻底失败，跳过此批数据。")
    return [] # 返回空列表，避免程序中断

# ================= 主程序 =================
def main():
    client = genai.Client(api_key=API_KEY)

    # 1. 读取所有数据到内存
    if not os.path.exists(INPUT_FILE):
        print(f"找不到文件: {INPUT_FILE}")
        return

    print(f"正在读取 {INPUT_FILE} ...")
    all_records = [] # 存储结构：{'original_id': 1, 'data': {...}}

    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        for index, line in enumerate(f):
            line = line.strip()
            if not line: continue
            try:
                # 预先解析 JSON，并绑定原始 ID
                json_obj = json.loads(line)
                all_records.append({
                    "original_id": index,
                    "data": json_obj
                })
            except:
                continue

    total_count = len(all_records)
    print(f"组装完成，共 {total_count} 条数据。")
    print(f"准备开始处理，每批 {BATCH_SIZE} 条，共需处理 {(total_count + BATCH_SIZE - 1) // BATCH_SIZE} 批。")

    # 2. 批量处理循环
    all_incorrect_items = []

    for i in range(0, total_count, BATCH_SIZE):
        batch = all_records[i : i + BATCH_SIZE]
        batch_num = (i // BATCH_SIZE) + 1
        
        print(f"\n🚀 正在处理第 {batch_num} 批 (ID: {batch[0]['original_id']} - {batch[-1]['original_id']})...")
        
        # 调用处理函数
        batch_errors = process_batch_with_retry(client, batch, batch_num)
        
        # 将结果加入总表
        if batch_errors:
            print(f"   ✅ 本批次发现 {len(batch_errors)} 个错误。")
            all_incorrect_items.extend(batch_errors)
        else:
            print("   ✅ 本批次未发现错误（或处理失败）。")

        # 批次间强制休息，保护 API 配额
        time.sleep(2) 

    # 3. 最终汇总与保存
    print("-" * 50)
    all_incorrect_items.sort(key=lambda x: x.id) # 按 ID 排序

    if not all_incorrect_items:
        print("🎉 检查全部完成，未发现任何错误！")
    else:
        print(f"检查全部完成！共发现 {len(all_incorrect_items)} 个错误。")
        
        # 转换为字典列表
        save_data = [item.model_dump() for item in all_incorrect_items]

        with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
            json.dump(save_data, f, ensure_ascii=False, indent=2)
        
        print(f"完整错误报告已保存至: {OUTPUT_FILE}")

if __name__ == "__main__":
    main()